SUPPORT VECTOR MACHINE


In [1]:
import sklearn
import numpy as np
import pandas as pd

In [7]:
train = pd.read_csv('resources/datasets/train.csv')
test = pd.read_csv('resources/datasets/test.csv')
train.head()

,Content,Label
0,every once in a while you see a film that is s...,pos
1,the love for family is one of the strongest dr...,pos
2,after the terminally bleak reservoir dogs and ...,pos
3,( warning to those who have not seen seven : ...,pos
4,"having not seen , "" who framed roger rabbit "" ...",pos


In [8]:
test.tail()

,Content,Label
195,my inner flag was at half-mast last year when ...,neg
196,"if anything , "" stigmata "" should be taken as ...",neg
197,woof ! too bad that leap of faith was the titl...,neg
198,the plot of big momma's house is martin lawren...,neg
199,"in the year 2029 , captain leo davidson ( mark...",neg


In [10]:
def getAnalysis(Label):
  if (Label == 'pos'):
    return '1'
  else:
    return '0'

train['rel'] = train['Label'].apply(getAnalysis)
test['rel'] = test['Label'].apply(getAnalysis)
test.tail()


,Content,Label,rel
195,my inner flag was at half-mast last year when ...,neg,0
196,"if anything , "" stigmata "" should be taken as ...",neg,0
197,woof ! too bad that leap of faith was the titl...,neg,0
198,the plot of big momma's house is martin lawren...,neg,0
199,"in the year 2029 , captain leo davidson ( mark...",neg,0


In [11]:
sum(train["rel"] == '0')

900

In [12]:
sum(train["rel"] == '1')

900

In [13]:
train.isnull().sum()

Content    0
Label      0
rel        0
dtype: int64

In [14]:
#install tweet-preprocessor to clean tweets
!pip install tweet-preprocessor

In [15]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [16]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [17]:
# clean training data
train_tweet = clean_tweets(train["Content"])
train_tweet = pd.DataFrame(train_tweet)

In [18]:
# append cleaned tweets to the training data
train["clean_tweet"] = train_tweet

# compare the cleaned and uncleaned tweets
train.head()


,Content,Label,rel,clean_tweet
0,every once in a while you see a film that is s...,pos,1,every once in a while you see a film that is s...
1,the love for family is one of the strongest dr...,pos,1,the love for family is one of the strongest dr...
2,after the terminally bleak reservoir dogs and ...,pos,1,after the terminally bleak reservoir dogs and ...
3,( warning to those who have not seen seven : ...,pos,1,warning to those who have not seen seven thi...
4,"having not seen , "" who framed roger rabbit "" ...",pos,1,having not seen who framed roger rabbit in ...


In [19]:
# clean the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["Content"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
test.tail()

,Content,Label,rel,clean_tweet
195,my inner flag was at half-mast last year when ...,neg,0,my inner flag was at half mast last year when ...
196,"if anything , "" stigmata "" should be taken as ...",neg,0,if anything stigmata should be taken as a w...
197,woof ! too bad that leap of faith was the titl...,neg,0,woof too bad that leap of faith was the title...
198,the plot of big momma's house is martin lawren...,neg,0,the plot of big mommas house is martin lawrenc...
199,"in the year 2029 , captain leo davidson ( mark...",neg,0,in the year captain leo davidson mark wahlbe...


In [20]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train.rel.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(train.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)


In [22]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

In [25]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  83.33333333333334 %
